In [ ]:
%matplotlib inline
import numpy as np
import os, sys
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path

import cv2

In [ ]:
img = cv2.imread('./test1.png')

img.shape

In [ ]:
def npre(l):
    
    for idx in range(len(l)):
        if not l[idx]:
            prefix = idx
            break
    for idx in range(len(l)):
        if not l[(-1) - idx]:
            return prefix, len(l)-idx

def ndiv(l, nd):
    (x1, x2), (y1, y2) = l
    nx, ny = nd
    xstep = round((x2-x1) / nx)
    ystep = round((y2-y1) / ny)
    xl, yl = [], []
    xpre = x1 - xstep
    for ro in range(nx):
#         print('xstep: ', xstep)
        xpre = xpre + xstep
        xl.append(xpre)
#         print('actual step: ', (x2 - xl[ro]) / (nx -ro+1))
        xstep =  round((x2 - xl[-1]) / (nx -ro))
#     print('xstep ', x2 - xpre)
    xl.append(x2)
    ypre = y1 - ystep
    for ro in range(ny):
#         print('ystep: ', ystep)
        ypre = ypre + ystep
        yl.append(ypre)
        ystep = round((y2 -yl[-1]) / (ny -ro))
#     print("ystep ", y2-ypre)
    yl.append(y2)
    return xl, yl

def genLine(xl,yl):
    for x in xl:
        yield (x, yl[0]), (x, yl[-1])
    for y in yl:
        yield (xl[0], y), (xl[-1], y)

def genRect(xl, yl):
    ydim = []
    for yidx in range(len(yl)-1):
        xdim = []
        for xidx in range(len(xl)-1):
            rect = np.array((xl[xidx], yl[yidx], xl[xidx+1], yl[yidx+1]))  #x1, y1, x2, y2
            x_step = (xl[xidx+1] - xl[xidx]) // 2
            y_step = (yl[yidx+1] - yl[yidx]) // 2
            across_rect = rect + np.array((x_step, 0, x_step, 0))
            y_across_rect = rect + np.array((0, y_step, 0, y_step))
            twi_across_rect = rect + np.array((x_step, y_step, x_step, y_step))
            xdim.append(np.array((rect, across_rect, y_across_rect, twi_across_rect)))
        ydim.append(xdim)
    return np.array(ydim)

def getColor(l):
    c = len(l)
    while True:
        c-=1
        yield l[c]
        if c == 0:
            c = len(l)
        
def showAnchor(i, c):
    bx,by,tx,ty=i
    x = [bx, bx, tx, tx, bx]
    y = [by, ty, ty, by, by]
    plt.plot(x, y, color=c)
    
def showAnchors(i, colors=['r', 'g', 'b']):
    cl = getColor(colors)
    cl = iter(cl)
    for item in i:
        showAnchor(item, next(cl))

def rect2mask(rects, size):
    mask = np.zeros(size, dtype=np.uint8)
    for rect in rects:
        bx,by,tx,ty=rect
        x = [bx, bx, tx, tx, bx]
        y = [by, ty, ty, by, by]
        poly = np.array([x, y], dtype=np.int32).T
        cv2.fillPoly(mask, [poly], (255,), 8, 0)
    return mask

def has_and(rect, img):
    x1, y1, x2, y2 = rect
    assert x1 < x2 and y1 < y2
    return img[y1:y2, x1:x2].max() > 0

def get_mask(size, rects, polys):
    poly_mask = np.zeros(size, dtype=np.uint8)
    for pidx in range(len(polys)-1):
        cv2.line(poly_mask, polys[pidx], polys[pidx+1], (255, ), thickness=7)
    rects_shape = rects.shape
    assert len(rects_shape) == 3
    rects = np.vstack(rects)
    mask = [has_and(r, poly_mask) for r in list(rects)]
    mask = np.array(mask)
    return mask.reshape(*(rects_shape[:2])), poly_mask
        

In [ ]:
hand = (79, 161), (36, 234), (105, 241), (128, 237)
person = (142, 106), (145, 120), (126, 133), (97, 126), (126,178), (145, 170)
print(hand, person)

In [ ]:
hand, person = np.array(hand).T, np.array(person).T


In [ ]:
h = img.transpose(2, 0, 1).min(axis=0).min(axis=0) == 255
w = img.transpose(2, 0, 1).min(axis=0).min(axis=1) == 255
(x1, x2), (y1, y2) = npre(h), npre(w)
xl,yl= ndiv((npre(h), npre(w)),(5, 5))
print(h.shape, w.shape, npre(h), npre(w))
print(xl,'\n', yl)

In [ ]:
rects =  genRect(xl, yl)
rects, across_rects, y_across_rects, twi_across_rects = rects[:,:,0,:], rects[:,:,1,:], rects[:,:,2,:], rects[:,:,3,:]
rect_mask, _ = get_mask((354, 230), rects, hand)
rect_mask = np.array([rect_mask])
print(rect_mask)
rect_mask =  rect_mask.repeat(4, axis=0)
rect_mask = rect_mask.transpose(1, 2, 0)
print(rect_mask.shape)

In [ ]:
rect_mask[...] = True
rect_mask

In [ ]:
print(rects.shape)
rect_list = np.where(rect_mask, rects, 0)
rect_list = np.vstack(rect_list)

across_rects_list, y_across_rects_list, twi_across_rects_list = np.where(rect_mask, across_rects, 0), np.where(rect_mask, y_across_rects, 0), np.where(rect_mask, twi_across_rects, 0)
across_rects_list, y_across_rects_list, twi_across_rects_list = across_rects_list[:, :-1], y_across_rects_list[:-1, :], twi_across_rects_list[:-1, :-1]
across_rects_list, y_across_rects_list, twi_across_rects_list = np.vstack(across_rects_list), np.vstack(y_across_rects_list), np.vstack(twi_across_rects_list)
print(rect_list.shape)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
showAnchors(rect_list)
showAnchors(across_rects_list, ['r'])
showAnchors(y_across_rects_list, ['b'])
showAnchors(twi_across_rects_list, ['g'])
# plt.plot(hand[0], hand[1], color='g')
# plt.plot(person[0], person[1], color='g')
plt.show()

In [ ]:
imgm = rect2mask(rect_list, (354, 230))
out = cv2.bitwise_and(img, np.array([imgm, imgm, imgm]).transpose(1, 2,0)) # img, mask=imgm)
plt.imshow(cv2.cvtColor(out, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
grid = list(genLine(xl, yl))
print(xl, yl)
# cv2.rectangle(img, pt1=(x1, y2), pt2=(x2, y1), color=(255, 0, 0), thickness=1 )
# for g in grid:
#     cv2.line(img, *g, color=(0, 255, 0), thickness=1 )
for g in grid:
    (x1, y1), (x2, y2) = g
    plt.plot([x1, x2], [y1, y2], color='g')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
test_rects =  genRect(xl, yl)
i1, polyimg = get_mask((354, 230), test_rects, hand)
plt.subplot(121)
showAnchors(np.vstack(rects))
plt.imshow(polyimg)
plt.subplot(122)
plt.imshow(i1)
plt.show()